In [ ]:
import pandas as pd
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 6.5 MB/s eta 0:00:00


In [ ]:
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.difference_update({'no', 'not'})

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import sklearn
import tqdm
from tqdm import tqdm
import nltk
import warnings
warnings.filterwarnings("ignore")
import cv2
from sklearn.model_selection import train_test_split
import PIL
from PIL import Image
import time

import tensorflow as tf
import keras
from keras.layers import Input,Dense,Conv2D,concatenate,Dropout,LSTM
from keras import Model
from tensorflow.keras import activations
import warnings
warnings.filterwarnings("ignore")
import nltk.translate.bleu_score as bleu

2024-07-18 04:32:55.819423: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 04:32:55.819576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 04:32:55.977576: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:


df2 = pd.read_csv('/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv')


In [ ]:
df2

,uid,filename,projection
0,1,1_IM-0001-4001.dcm.png,Frontal
1,1,1_IM-0001-3001.dcm.png,Lateral
2,2,2_IM-0652-1001.dcm.png,Frontal
3,2,2_IM-0652-2001.dcm.png,Lateral
4,3,3_IM-1384-1001.dcm.png,Frontal
...,...,...,...
7461,3997,3997_IM-2048-1002.dcm.png,Lateral
7462,3998,3998_IM-2048-1001.dcm.png,Frontal
7463,3998,3998_IM-2048-1002.dcm.png,Lateral
7464,3999,3999_IM-2049-1001.dcm.png,Frontal


In [ ]:
df2 = df2.drop_duplicates(subset=['uid', 'projection'])
df2

,uid,filename,projection
0,1,1_IM-0001-4001.dcm.png,Frontal
1,1,1_IM-0001-3001.dcm.png,Lateral
2,2,2_IM-0652-1001.dcm.png,Frontal
3,2,2_IM-0652-2001.dcm.png,Lateral
4,3,3_IM-1384-1001.dcm.png,Frontal
...,...,...,...
7461,3997,3997_IM-2048-1002.dcm.png,Lateral
7462,3998,3998_IM-2048-1001.dcm.png,Frontal
7463,3998,3998_IM-2048-1002.dcm.png,Lateral
7464,3999,3999_IM-2049-1001.dcm.png,Frontal


In [ ]:
directory_path = r'/kaggle/input/chest-xrays-indiana-university/images/images_normalized'

df2['filename'] = df2['filename'].apply(lambda x: f'{directory_path}/{x}')

In [ ]:
pivot_df = df2.pivot(index='uid', columns='projection', values='filename').reset_index()
pivot_df

projection,uid,Frontal,Lateral
0,1,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...
1,2,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...
2,3,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...
3,4,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...
4,5,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...
...,...,...,...
3846,3995,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...
3847,3996,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...
3848,3997,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...
3849,3998,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...


In [ ]:
df1 = pd.read_csv('/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv')

In [ ]:
df1

,uid,MeSH,Problems,image,indication,comparison,findings,impression
0,1,normal,normal,Xray Chest PA and Lateral,Positive TB test,None.,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,Cardiomegaly/borderline;Pulmonary Artery/enlarged,Cardiomegaly;Pulmonary Artery,"Chest, 2 views, frontal and lateral",Preop bariatric surgery.,None.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,normal,normal,Xray Chest PA and Lateral,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...","Pulmonary Disease, Chronic Obstructive;Bullous...","PA and lateral views of the chest XXXX, XXXX a...",XXXX-year-old XXXX with XXXX.,None available,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,Osteophyte/thoracic vertebrae/multiple/small;T...,Osteophyte;Thickening;Lung,Xray Chest PA and Lateral,Chest and nasal congestion.,NaN,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.
...,...,...,...,...,...,...,...,...
3846,3995,Lung/hyperdistention/mild;Diaphragm/bilateral/...,Lung;Diaphragm;Cicatrix;Pulmonary Atelectasis;...,Xray Chest PA and Lateral,"Nausea, vomiting x2 weeks. Dialysis patient.","XXXX, XXXX.",The cardiomediastinal silhouette and pulmonary...,1. Interval resolution of bibasilar airspace d...
3847,3996,Spine/degenerative,Spine,Xray Chest PA and Lateral,NaN,None.,The lungs are clear. Heart size is normal. No ...,Clear lungs. No acute cardiopulmonary abnormal...
3848,3997,Opacity/lung/upper lobe/right/round/small;Gran...,Opacity;Granuloma,PA and lateral views of the chest.,XXXX-year-old male with positive PPD.,None available.,"Heart size within normal limits. Small, nodula...","No acute findings, no evidence for active TB."
3849,3998,normal,normal,"PA and lateral chest XXXX, XXXX XXXX comparis...",tuberculosis positive PPD,NaN,NaN,Heart size is normal and the lungs are clear.


In [ ]:
df1['findings'][6]

'The cardiac contours are normal. XXXX basilar atelectasis. The lungs are clear. Thoracic spondylosis. Lower cervical XXXX arthritis.'

In [ ]:
columns_to_combine = ['MeSH', 'image', 'comparison']

In [ ]:
df1.drop(columns=columns_to_combine, inplace=True)

In [ ]:
df1

,uid,Problems,indication,findings,impression
0,1,normal,Positive TB test,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.
1,2,Cardiomegaly;Pulmonary Artery,Preop bariatric surgery.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.
2,3,normal,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,"No displaced rib fractures, pneumothorax, or p..."
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...",XXXX-year-old XXXX with XXXX.,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...
4,5,Osteophyte;Thickening;Lung,Chest and nasal congestion.,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.
...,...,...,...,...,...
3846,3995,Lung;Diaphragm;Cicatrix;Pulmonary Atelectasis;...,"Nausea, vomiting x2 weeks. Dialysis patient.",The cardiomediastinal silhouette and pulmonary...,1. Interval resolution of bibasilar airspace d...
3847,3996,Spine,NaN,The lungs are clear. Heart size is normal. No ...,Clear lungs. No acute cardiopulmonary abnormal...
3848,3997,Opacity;Granuloma,XXXX-year-old male with positive PPD.,"Heart size within normal limits. Small, nodula...","No acute findings, no evidence for active TB."
3849,3998,normal,tuberculosis positive PPD,NaN,Heart size is normal and the lungs are clear.


In [ ]:
df1["Problems"][6]

'Pulmonary Atelectasis;Spondylosis;Arthritis'

In [ ]:
# Define a function to create the sentence-formatted report
def create_sentence_report(row):
    return f"""{row['findings']}{row['impression']}"""

In [ ]:
# Apply the function to each row and create a new column 'sentence_report'
df1['report'] = df1.apply(create_sentence_report, axis=1)

In [ ]:
df1["report"][6]

'The cardiac contours are normal. XXXX basilar atelectasis. The lungs are clear. Thoracic spondylosis. Lower cervical XXXX arthritis.Basilar atelectasis. No confluent lobar consolidation or pleural effusion.'

In [ ]:
import contractions

In [ ]:
def expand_contractions(text):
    contractions = {
        "isn't": "is not",
        "doesn't": "does not",
        # Add more contractions as needed
    }
    for contraction, expanded in contractions.items():
        text = re.sub(r'\b{}\b'.format(contraction), expanded, text)
    return text

In [ ]:
def preprocess_text(text):
    # Define stopwords
    stop_words = set(stopwords.words('english'))

    # Convert to lowercase
    text = text.lower()

    # Expand contractions
    text = contractions.fix(text)

    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Remove non-alphabetical characters (except spaces and periods)
    text = re.sub(r'[^a-zA-Z\s.]', '', text)


    # Remove words containing 'xx' or similar patterns
    text = re.sub(r'\bxx+\w*\b', '', text)
    text = re.sub(r'\b\w*xx+\w*\b', '', text)

    # Remove the phrase 'year old'
    text = re.sub(r'\byear old\b', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove stopwords and filter out words shorter than three characters
    text = ' '.join([word for word in text.split() if word not in stop_words and len(word) > 2])

    return text

In [ ]:
df1['report'] = df1['report'].apply(preprocess_text)

In [ ]:
df1['report'][1]

'borderline cardiomegaly. midline sternotomy enlarged pulmonary arteries. clear lungs. inferior .no acute pulmonary findings.'

In [ ]:
df1

,uid,Problems,indication,findings,impression,report
0,1,normal,Positive TB test,The cardiac silhouette and mediastinum size ar...,Normal chest x-XXXX.,cardiac silhouette mediastinum size within nor...
1,2,Cardiomegaly;Pulmonary Artery,Preop bariatric surgery.,Borderline cardiomegaly. Midline sternotomy XX...,No acute pulmonary findings.,borderline cardiomegaly. midline sternotomy en...
2,3,normal,"rib pain after a XXXX, XXXX XXXX steps this XX...",NaN,"No displaced rib fractures, pneumothorax, or p...",nanno displaced rib fractures pneumothorax ple...
3,4,"Pulmonary Disease, Chronic Obstructive;Bullous...",XXXX-year-old XXXX with XXXX.,There are diffuse bilateral interstitial and a...,1. Bullous emphysema and interstitial fibrosis...,diffuse bilateral interstitial alveolar opacit...
4,5,Osteophyte;Thickening;Lung,Chest and nasal congestion.,The cardiomediastinal silhouette and pulmonary...,No acute cardiopulmonary abnormality.,cardiomediastinal silhouette pulmonary vascula...
...,...,...,...,...,...,...
3846,3995,Lung;Diaphragm;Cicatrix;Pulmonary Atelectasis;...,"Nausea, vomiting x2 weeks. Dialysis patient.",The cardiomediastinal silhouette and pulmonary...,1. Interval resolution of bibasilar airspace d...,cardiomediastinal silhouette pulmonary vascula...
3847,3996,Spine,NaN,The lungs are clear. Heart size is normal. No ...,Clear lungs. No acute cardiopulmonary abnormal...,lungs clear. heart size normal. pneumothorax. ...
3848,3997,Opacity;Granuloma,XXXX-year-old male with positive PPD.,"Heart size within normal limits. Small, nodula...","No acute findings, no evidence for active TB.",heart size within normal limits. small nodular...
3849,3998,normal,tuberculosis positive PPD,NaN,Heart size is normal and the lungs are clear.,nanheart size normal lungs clear.


In [ ]:
columns_to_combine = ['Problems', 'indication', 'findings','impression']

In [ ]:
df1.drop(columns=columns_to_combine, inplace=True)

In [ ]:
from tensorflow.keras.applications import DenseNet121

image_shape= (224,224,3)
image_input= Input(shape=(224,224,3))

base=DenseNet121(include_top=False,input_tensor=image_input,input_shape=image_shape)
pred=Dense(14,"sigmoid")(base.output)

chexnet_model=Model(inputs=base.input,outputs=pred)
chexnet_model.load_weights("/kaggle/input/chexnet-weights/CheXNet weights.h5")

chexnet_model.summary()

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 56, 56,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 56, 56,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 56, 56,    │     12,288 │ conv2_block2_0_r

 Total params: 7,051,854 (26.90 MB)

 Trainable params: 6,968,206 (26.58 MB)

 Non-trainable params: 83,648 (326.75 KB)

In [ ]:
final_chexnet_model=Model(inputs=chexnet_model.inputs,outputs=chexnet_model.layers[-2].output,name="Chexnet_model")


In [ ]:
final_chexnet_model.save("/kaggle/working/chexnet.h5")

In [ ]:
data = pd.merge(pivot_df, df1, on='uid', how='inner')

In [ ]:
data

,uid,Frontal,Lateral,report
0,1,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,cardiac silhouette mediastinum size within nor...
1,2,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,borderline cardiomegaly. midline sternotomy en...
2,3,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,nanno displaced rib fractures pneumothorax ple...
3,4,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,diffuse bilateral interstitial alveolar opacit...
4,5,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,cardiomediastinal silhouette pulmonary vascula...
...,...,...,...,...
3846,3995,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,cardiomediastinal silhouette pulmonary vascula...
3847,3996,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,lungs clear. heart size normal. pneumothorax. ...
3848,3997,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,heart size within normal limits. small nodular...
3849,3998,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,nanheart size normal lungs clear.


In [ ]:
data.dropna(inplace=True)

In [ ]:
data

,uid,Frontal,Lateral,report
0,1,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,cardiac silhouette mediastinum size within nor...
1,2,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,borderline cardiomegaly. midline sternotomy en...
2,3,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,nanno displaced rib fractures pneumothorax ple...
3,4,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,diffuse bilateral interstitial alveolar opacit...
4,5,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,cardiomediastinal silhouette pulmonary vascula...
...,...,...,...,...
3846,3995,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,cardiomediastinal silhouette pulmonary vascula...
3847,3996,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,lungs clear. heart size normal. pneumothorax. ...
3848,3997,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,heart size within normal limits. small nodular...
3849,3998,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,nanheart size normal lungs clear.


In [ ]:
def image_feature_extraction(image1,image2):


  image_1 = Image.open(image1)
  image_1= np.asarray(image_1.convert("RGB"))


  image_2=Image.open(image2)
  image_2 = np.asarray(image_2.convert("RGB"))

    #normalize the values of the image
  image_1=image_1/255
  image_2=image_2/255

    #resize all image into (224,224)
  image_1 = cv2.resize(image_1,(224,224))
  image_2 = cv2.resize(image_2,(224,224))

  image_1= np.expand_dims(image_1, axis=0)
  image_2= np.expand_dims(image_2, axis=0)

    #now we have read two image per patient. this is goven to the chexnet model for feature extraction

  image_1_out=final_chexnet_model(image_1)
  image_2_out=final_chexnet_model(image_2)
  #conactenate along the width
  conc=np.concatenate((image_1_out,image_2_out),axis=2)
  #reshape into(no.of images passed, length*breadth, depth)
  image_feature=tf.reshape(conc, (conc.shape[0], -1, conc.shape[-1]))



  return image_feature


In [ ]:
train_size = int(0.8 * len(data))  # 80% for training, adjust as needed
test_size = len(data) - train_size

In [ ]:
train = data.iloc[:train_size]
test = data.iloc[train_size:]


In [ ]:
test

,uid,Frontal,Lateral,report
3099,3226,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,cardiac contours normal. lungs clear. thoracic...
3100,3227,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,cardiomediastinal silhouette within normal lim...
3101,3228,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,heart normal size. mediastinum unremarkable. b...
3102,3229,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,heart size within normal limits. trachea midli...
3103,3230,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,low lung volumes patient rotation. given diffe...
...,...,...,...,...
3846,3995,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,cardiomediastinal silhouette pulmonary vascula...
3847,3996,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,lungs clear. heart size normal. pneumothorax. ...
3848,3997,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,heart size within normal limits. small nodular...
3849,3998,/kaggle/input/chest-xrays-indiana-university/i...,/kaggle/input/chest-xrays-indiana-university/i...,nanheart size normal lungs clear.


In [ ]:
train_features=np.load("/kaggle/input/image-features-data-gru/train_features_GRU.npy")
test_features=np.load("/kaggle/input/image-features-data-gru/test_features_GRU.npy")

In [ ]:
train_features.shape

(2710, 98, 1024)

In [ ]:
# data_features=np.load("/kaggle/input/image-feaatures-augmented/data_features.npy")

In [ ]:
# train_features = np.vstack((train_features, data_features))

In [ ]:
train_features.shape

(2710, 98, 1024)

In [ ]:
k=190
print(test_features[k])
one=test.iloc[k]["Frontal"]
two=test.iloc[k]["Lateral"]
print(image_feature_extraction(one,two))

[[0.00000000e+00 1.06397609e-03 0.00000000e+00 ... 0.00000000e+00
  9.21273053e-01 0.00000000e+00]
 [0.00000000e+00 1.00856391e-03 3.56457429e-03 ... 0.00000000e+00
  9.37728524e-01 0.00000000e+00]
 [0.00000000e+00 1.70407351e-03 1.40303746e-03 ... 0.00000000e+00
  9.37571406e-01 0.00000000e+00]
 ...
 [0.00000000e+00 2.92105135e-03 8.71939585e-04 ... 0.00000000e+00
  9.77788568e-01 0.00000000e+00]
 [0.00000000e+00 1.97018939e-03 0.00000000e+00 ... 0.00000000e+00
  9.25342381e-01 0.00000000e+00]
 [0.00000000e+00 1.09073566e-03 1.46924565e-03 ... 0.00000000e+00
  9.00472760e-01 0.00000000e+00]]
tf.Tensor(
[[[0.0000000e+00 1.0639750e-03 0.0000000e+00 ... 0.0000000e+00
   9.2127305e-01 0.0000000e+00]
  [0.0000000e+00 1.0085623e-03 3.5645780e-03 ... 0.0000000e+00
   9.3772835e-01 0.0000000e+00]
  [0.0000000e+00 1.7040754e-03 1.4030356e-03 ... 0.0000000e+00
   9.3757129e-01 0.0000000e+00]
  ...
  [0.0000000e+00 2.9210523e-03 8.7194610e-04 ... 0.0000000e+00
   9.7778875e-01 0.0000000e+00]
  [

In [ ]:
print(train_features.shape)
print(test_features.shape)

(2710, 98, 1024)
(678, 98, 1024)


In [ ]:
# reports=pd.read_csv('/kaggle/input/augmented-reports/reports.csv')

In [ ]:
# train_reports=train["report"]

In [ ]:
# train_reports=pd.DataFrame(train_reports)

In [ ]:
# train_reports=pd.concat([train_reports, reports], ignore_index=True)

In [ ]:
train_report=["<sos> "+text+" <eos>" for text in train["report"].values]
train_report_in=["<sos> "+text for text in train["report"].values]
train_report_out=[text+" <eos>" for text in train["report"].values]

test_report=["<sos> " +text+" <eos>" for text in test["report"].values]
test_report_in=["<sos> " +text for text in test["report"].values]
test_report_out=[text+" <eos>" for text in test["report"].values]

In [ ]:
print(train_report_in[4])
print("*"*100)
print(train_report_out[4])


<sos> cardiomediastinal silhouette pulmonary vasculature within normal limits. pneumothorax pleural effusion. focal areas consolidation. cholecystectomy clips present. small tspine osteophytes. biapical pleural thickening unchanged prior. mildly hyperexpanded lungs.no acute cardiopulmonary abnormality.
****************************************************************************************************
cardiomediastinal silhouette pulmonary vasculature within normal limits. pneumothorax pleural effusion. focal areas consolidation. cholecystectomy clips present. small tspine osteophytes. biapical pleural thickening unchanged prior. mildly hyperexpanded lungs.no acute cardiopulmonary abnormality. <eos>


In [ ]:
bs=10
max_len=80


In [ ]:
train_report[6]

'<sos> cardiac contours normal. basilar atelectasis. lungs clear. thoracic spondylosis. lower cervical arthritis.basilar atelectasis. confluent lobar consolidation pleural effusion. <eos>'

In [ ]:
token=tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;?@[\]^_{|}~\t\n')

token.fit_on_texts(train_report)
vocab_size=len(token.word_index)+1

seq=token.texts_to_sequences(train_report_in)
train_padded_inp=tf.keras.preprocessing.sequence.pad_sequences(seq,maxlen=max_len,padding="post")


seq=token.texts_to_sequences(train_report_out)
train_padded_out=tf.keras.preprocessing.sequence.pad_sequences(seq,maxlen=max_len,padding="post")

seq=token.texts_to_sequences(test_report_in)
test_padded_inp=tf.keras.preprocessing.sequence.pad_sequences(seq,maxlen=max_len,padding="post")


seq=token.texts_to_sequences(test_report_out)
test_padded_out=tf.keras.preprocessing.sequence.pad_sequences(seq,maxlen=max_len,padding="post")

In [ ]:
vocab_size

1809

In [ ]:
# import pickle

In [ ]:
import json

# Save the tokenizer configuration and word index
tokenizer_json = token.to_json()
with open('tokenizer.json', 'w') as file:
    file.write(json.dumps(tokenizer_json))


In [ ]:
token.index_word[2]

'<sos>'

In [ ]:
token.word_index['<sos>']

2

In [ ]:
embeddings_index=dict()
f = open('/kaggle/input/glove-txt/glove.6B.300d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print("Done")
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tqdm(token.word_index.items()):
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector


Done


100%|██████████| 1808/1808 [00:00<00:00, 184854.88it/s]


In [ ]:
np.save('embedding_matrix.npy', embedding_matrix)

In [ ]:
enc_units=64
embedding_dim=300
dec_units=64
att_units=64


In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.77691001, -0.038156  , -0.20552   , ..., -0.20263   ,
         0.27416   ,  0.11023   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.0075101 , -0.23317   ,  0.36638999, ..., -0.025231  ,
        -0.34209999,  0.22964001],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.12775999,  0.52021998, -0.04902   , ...,  0.007387  ,
         0.32683   , -0.32311001]])

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        self.dense1 = tf.keras.layers.Dense(self.units, activation="relu", name="encoder_dense")
        self.maxpool = tf.keras.layers.Dropout(0.5)

    def call(self, input_):
        enc_out = self.maxpool(input_)
        enc_out = self.dense1(enc_out)
        return enc_out

    def initialize_states(self, batch_size):
        forward_h = tf.zeros((batch_size, self.units))
        back_h = tf.zeros((batch_size, self.units))
        return forward_h, back_h

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'units': self.units
        })
        return config


In [ ]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, att_units):
        super().__init__()
        self.att_units = att_units

    def build(self, input_shape):
        self.wa = tf.keras.layers.Dense(self.att_units)
        self.wb = tf.keras.layers.Dense(self.att_units)
        self.v = tf.keras.layers.Dense(1)

    def call(self, decoder_hidden_state, encoder_output):
        x = tf.expand_dims(decoder_hidden_state, 1)
        alpha_dash = self.v(tf.nn.tanh(self.wa(encoder_output) + self.wb(x)))
        alphas = tf.nn.softmax(alpha_dash, 1)
        context_vector = tf.matmul(encoder_output, alphas, transpose_a=True)[:,:,0]
        return context_vector, alphas

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'att_units': self.att_units
        })
        return config

In [ ]:
class One_Step_Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, input_length, dec_units, att_units):
        super().__init__()
        self.att_units = att_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.attention = Attention(self.att_units)
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, output_dim=self.embedding_dim,
                                                   input_length=self.input_length, mask_zero=True, trainable=False, weights=[embedding_matrix])
        self.gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True))
        self.dense = tf.keras.layers.Dense(self.vocab_size, name="decoder_final_dense")
        self.dense_2 = tf.keras.layers.Dense(self.embedding_dim, name="decoder_dense2")
        self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, input_to_decoder, encoder_output, for_h, bac_h):
        embed = self.embedding(input_to_decoder)
        state_h = tf.keras.layers.Add()([for_h, bac_h])
        context_vector, alpha = self.attention(state_h, encoder_output)
        context_vector = self.dense_2(context_vector)
        result = tf.concat([tf.expand_dims(context_vector, axis=1), embed], axis=-1)
        output, forward_h, back_h = self.gru(result, initial_state=[for_h, bac_h])
        out = tf.reshape(output, (-1, output.shape[-1]))
        out = self.dropout(out)
        dense_op = self.dense(out)
        return dense_op, forward_h, back_h, alpha

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'vocab_size': self.vocab_size,
            'embedding_dim': self.embedding_dim,
            'input_length': self.input_length,
            'dec_units': self.dec_units,
            'att_units': self.att_units
        })
        return config

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, output_length, dec_units, att_units):
        super().__init__()
        self.onestep = One_Step_Decoder(vocab_size, embedding_dim, output_length, dec_units, att_units)

    def call(self, input_to_decoder, encoder_output, state_1, state_2):
        all_outputs = tf.TensorArray(tf.float32, input_to_decoder.shape[1], name="output_array")
        for step in range(input_to_decoder.shape[1]):
            output, state_1, state_2, alpha = self.onestep(input_to_decoder[:, step:step+1], encoder_output, state_1, state_2)
            all_outputs = all_outputs.write(step, output)
        all_outputs = tf.transpose(all_outputs.stack(), [1, 0, 2])
        return all_outputs

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'vocab_size': self.onestep.vocab_size,
            'embedding_dim': self.onestep.embedding_dim,
            'output_length': self.onestep.input_length,
            'dec_units': self.onestep.dec_units,
            'att_units': self.onestep.att_units
        })
        return config


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
class encoder_decoder(tf.keras.Model):
    def __init__(self, enc_units, embedding_dim, vocab_size, output_length, dec_units, att_units, batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.encoder = Encoder(enc_units)
        self.decoder = Decoder(vocab_size, embedding_dim, output_length, dec_units, att_units)

    def call(self, data):
        features, report = data[0], data[1]
        encoder_output = self.encoder(features)
        state_h, back_h = self.encoder.initialize_states(self.batch_size)
        output = self.decoder(report, encoder_output, state_h, back_h)
        return output

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'enc_units': self.encoder.units,
            'embedding_dim': self.decoder.onestep.embedding_dim,
            'vocab_size': self.decoder.onestep.vocab_size,
            'output_length': self.decoder.onestep.input_length,
            'dec_units': self.decoder.onestep.dec_units,
            'att_units': self.decoder.onestep.att_units,
            'batch_size': self.batch_size
        })
        return config

In [ ]:
model  = encoder_decoder(enc_units,embedding_dim,vocab_size,max_len,dec_units,att_units,bs)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def custom_lossfunction(y_true, y_pred):
    #getting mask value
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))

    #calculating the loss
    loss_ = loss_function(y_true, y_pred)

    #converting mask dtype to loss_ dtype
    mask = tf.cast(mask, dtype=loss_.dtype)

    #applying the mask to loss
    loss_ = loss_*mask

    #getting mean over all the values
    loss_ = tf.reduce_mean(loss_)
    return loss_

In [ ]:
model.compile(optimizer=optimizer,loss=custom_lossfunction)

In [ ]:
red_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",factor=0.2,patience=2, min_lr=0.0001)
ckpt=tf.keras.callbacks.ModelCheckpoint("model2wts/ckpt.keras",monitor='val_loss', verbose=0, save_best_only=True,save_weights_only=False, mode='auto')


In [ ]:
train_features.shape

(2710, 98, 1024)

In [ ]:
test_features.shape

(678, 98, 1024)

In [ ]:
model.fit([train_features[:2710],train_padded_inp[:2710]],train_padded_out[:2710],validation_data=([test_features[:670],test_padded_inp[:670]],test_padded_out[:670]),
          batch_size=bs,epochs=15,callbacks=[red_lr,ckpt])

Epoch 1/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 375s 383ms/step - loss: 1.8552 - val_loss: 1.2247 - learning_rate: 0.0010
Epoch 2/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 65s 240ms/step - loss: 1.2117 - val_loss: 1.0114 - learning_rate: 0.0010
Epoch 3/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 65s 239ms/step - loss: 1.0430 - val_loss: 0.9356 - learning_rate: 0.0010
Epoch 4/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 64s 238ms/step - loss: 0.9701 - val_loss: 0.8883 - learning_rate: 0.0010
Epoch 5/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 65s 240ms/step - loss: 0.8686 - val_loss: 0.8576 - learning_rate: 0.0010
Epoch 6/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 65s 241ms/step - loss: 0.8313 - val_loss: 0.8365 - learning_rate: 0.0010
Epoch 7/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 65s 240ms/step - loss: 0.8028 - val_loss: 0.8222 - learning_rate: 0.0010
Epoch 8/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 65s 241ms/step - loss: 0.7586 - val_loss: 0.8163 - learning_rate: 0.0010
Epoch 9/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 65s 241ms/step - loss: 0.7204 - val_loss: 0.8050 - lear

In [ ]:
model.load_weights("/kaggle/working/model2wts/ckpt.keras")

In [ ]:
model.save("/kaggle/working/grumodel_dense.h5")

In [ ]:
def take_second(elem):
    return elem[1]

In [ ]:

import time

In [ ]:
import random
import time
import cv2

def beam_search(image1, image2, beam_index):
    hidden_state = tf.zeros((1, enc_units))
    hidden_state2 = tf.zeros((1, enc_units))
    image_features = image_feature_extraction(image1, image2)

    encoder_out = model.layers[0](image_features)

    start_token = [token.word_index["<sos>"]]
    dec_word = [[start_token, 0.0]]
    while len(dec_word[0][0]) < max_len:
        temp = []
        for word in dec_word:
            predict, hidden_state, hidden_state2, alpha = model.layers[1].onestep(tf.expand_dims([word[0][-1]], 1), encoder_out, hidden_state, hidden_state2)
            word_predict = np.argsort(predict[0])[-beam_index:]
            for i in word_predict:
                next_word, probab = word[0][:], word[1]
                next_word.append(i)
                probab += predict[0][i]
                temp.append([next_word, probab.numpy()])
        dec_word = temp
        dec_word = sorted(dec_word, key=take_second)
        dec_word = dec_word[-beam_index:]

    final = dec_word[-1]
    report = final[0]
    score = final[1]
    temp = []

    for word in report:
        if word != 0:
            if word != token.word_index["<eos>"]:
                temp.append(token.index_word[word])
            else:
                break

    rep = " ".join(e for e in temp)
    return rep, score


In [ ]:
import random

In [ ]:
import random
start=time.time()
i=random.sample(range(test.shape[0]),1)[0]
img1=test.iloc[i]["Frontal"]
img2=test.iloc[i]["Lateral"]
  #show th corresponding x-ray images
i1=cv2.imread(img1)
i2=cv2.imread(img2)
plt.figure(figsize=(10,6))
plt.subplot(131)
plt.title("image1")
plt.imshow(i1)
plt.subplot(132)
plt.title("image2")
plt.imshow(i2)
plt.show()
  #printing the actual and generated results

result,score=beam_search(img1,img2,4)
actual=test_report[i]

print("ACTUAL REPORT: ",actual)
print("GENERATED REPORT: ",result)
end=time.time()
print("BLEU SCORE IS: ",bleu.sentence_bleu(actual,result))
print("time required for the evaluation is ",end-start)

In [ ]:
# def evaluate_reports(test_data, beam_index):
#     actual_reports = []
#     generated_reports = []
#     bleu_scores = []

#     for i in tqdm(range(len(test_data))):
#         img1 = test_data.iloc[i]["Frontal"]
#         img2 = test_data.iloc[i]["Lateral"]

#         # Generate report
#         result, _ = beam_search(img1, img2, beam_index)
#         actual = test_report[i]

#         # Calculate BLEU score
#         bleu_score = bleu.sentence_bleu([actual.split()], result.split())

#         # Store results
#         actual_reports.append(actual)
#         generated_reports.append(result)
#         bleu_scores.append(bleu_score)

#     # Create a DataFrame to store the results
#     results_df = pd.DataFrame({
#         'Actual Report': actual_reports,
#         'Generated Report': generated_reports,
#         'BLEU Score': bleu_scores
#     })

#     return results_df


In [ ]:
# results_df = evaluate_reports(test, beam_index=3)

In [ ]:
# results_df.to_csv('report_evaluation.csv', index=False)